In [ ]:
# 这是用RNN模型将sin的输入转换为cos的输出
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt


In [ ]:

TIME_STEP = 5                               # rnn 定义的时间步数
INPUT_SIZE = 1                              # 因为输入的是1个数字，也就是1个feature
LR = 0.02                                   # learning rate
HIDDEN_SIZE = 64                            # 隐藏层的特征数量
torch.manual_seed(1)                        # 随机数种子


In [67]:

class RNN(nn.Module):                                   # hidden = np.tanh(np.dot(self.W_hh, hidden) + np.dot(self.W_xh, x))
    def __init__(self):
        super().__init__()
        self.rnn = nn.RNN(                              # 自己定义一个RNN神经网络
            input_size = INPUT_SIZE,                    # 输入 x 的feature 维度
            hidden_size = HIDDEN_SIZE,                  # 隐状态 hidden_state 中的feature维度
            num_layers = 3,                             # RNN 的层数
            nonlinearity='relu',                        # 指定激活函数 [‘tanh’ | ’relu’]. 默认: ‘tanh’
            bias = True,                                # 如果是 False , 那么 RNN 层就不会使用偏置权重 b_ih 和 b_hh, 默认: True
            batch_first = True,                         # 如果 True, 输入Tensor的shape应该是(batch, seq, features),并且输出也是一样.
            dropout = 0,                                # 如果值非零, 那么除了最后一层外, 其它层的输出都会套上一个 dropout 层
            bidirectional = False                       # 如果 True , 将会变成一个双向 RNN, 默认为 False
        )
        self.out = nn.Linear(HIDDEN_SIZE, 1)            # 定义一个输出层，这是RNN的最后输出，只用输出output_vector

    def forward(self, x, h_state):                      # 这就是RNN每次输入的参数x和h
        # x (batch, time_step, input_size)              # 这是RNN的ｘ的维度            (批量, 序列长度, 输入的特征维度）
        # h_state (n_layers, batch, hidden_size)        # 这是hidden_state的维度       (层数×方向, 批量, 输出的特征维度）/*方向：单向是１；双向是２*/
        # r_out (batch, time_step, hidden_size)         # 这是网络实际输出的r_out的维度 (批量，序列长度，输出的特征维度X方向）
        r_out, h_state = self.rnn(x, h_state)           # RNN每次输入x, hidden_state; 输出r_out, hidden_state;
        outs = []
        for time_step in range(r_out.size(1)):
            outs.append(self.out(r_out[:, time_step, :]))

        return torch.stack(outs, dim=1), h_state        # RNN的forward输出了output_vector, hidden_state

In [68]:

rnn = RNN()
optimizer = torch.optim.Adam(rnn.parameters(), lr=LR, weight_decay=1e-6)
loss_func = nn.MSELoss()

In [70]:

h_state = None                                                          # 起始时输入给RNN的hidden_state就是None

for step in range(200):                                                 # 计算２００次。相当于２００个顺序的时间片数据丢进去计算
    start, end = step*np.pi, (step+1)*np.pi                             # 设计一小段数据起始点
    steps = np.linspace(start, end, TIME_STEP, dtype=np.float32)        # 生成一小段数据
    x_np = np.sin(steps)                                                # 这就是用来输入的数据
    y_np = np.cos(steps)                                                # 这就是需要被预测的数据
    x = Variable(torch.from_numpy(x_np[np.newaxis, :, np.newaxis]))     # shape 1D -> 3D
    y = Variable(torch.from_numpy(y_np[np.newaxis, :, np.newaxis]))
    # print(x.size())
    # print(y.size())
    prediction, h_state = rnn(x, h_state)                               # 这就是一次RNN训练出来的结果
    h_state = Variable(h_state.data)                                    # 把tensor中的数据取出来

    loss = loss_func(prediction, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print("step:{}  loss:{:.9f}".format(step, loss.data.float()))

#     plt.ion()                                                           # 把实际图和预测图动态打印出来
#     plt.plot(steps, y_np, color='b')
#     plt.plot(steps, np.squeeze(prediction.data.numpy()), color='r')
#     plt.show()
#     plt.pause(0.30)

step:0  loss:0.769753754
step:1  loss:0.018608779
step:2  loss:0.196479589
step:3  loss:0.071586773
step:4  loss:0.012409372
step:5  loss:0.044172186
step:6  loss:0.158242732
step:7  loss:0.023251805
step:8  loss:0.023255471
step:9  loss:0.033692829
step:10  loss:0.047859583
step:11  loss:0.024848435
step:12  loss:0.027895218
step:13  loss:0.020639541
step:14  loss:0.021983327
step:15  loss:0.019498849
step:16  loss:0.027365714
step:17  loss:0.032652780
step:18  loss:0.015358785
step:19  loss:0.008105570
step:20  loss:0.013356498
step:21  loss:0.013916977
step:22  loss:0.029352468
step:23  loss:0.014032751
step:24  loss:0.003490339
step:25  loss:0.001934425
step:26  loss:0.025918942
step:27  loss:0.002195222
step:28  loss:0.008604687
step:29  loss:0.001199994
step:30  loss:0.008275824
step:31  loss:0.004451863
step:32  loss:0.015970895
step:33  loss:0.005393638
step:34  loss:0.005837458
step:35  loss:0.002005154
step:36  loss:0.002670521
step:37  loss:0.004802625
step:38  loss:0.010009